### Read TF list and ensembl to symbol mapping file

In [6]:
import pandas as pd
tf_list = pd.read_csv("data/tf_list/tf_list.csv", index_col = 0)    
ensembl_to_symbol = pd.read_csv("data/ensembl_to_symbol.csv",index_col = 0)
ensembl_to_symbol = ensembl_to_symbol.loc[~ensembl_to_symbol["ensembl_id"].duplicated(),:]
ensembl_to_symbol.index = ensembl_to_symbol["ensembl_id"]
ensembl_to_symbol.index.name = None

### Load expression data, and TF activitiy scores.

In [3]:
import h5py
import numpy as np
import pandas as pd

# Specify the tissue name here
tissue = 'Liver'

# First we get indices of TFs
with h5py.File('data/processed/rpkm.hdf5', 'r') as f:
    col_names = np.array(f[tissue]['ensembl']).astype(str)
tf_idx = np.where(np.isin(col_names, tf_list["Ensembl ID"].values))[0]

# Get expressions only for TF columns. RPKM values are log10 transformed. 
with h5py.File('data/processed/rpkm.hdf5', 'r') as f:
    rpkm = pd.DataFrame(np.log10(np.array(f[tissue]['exp'][:,tf_idx])+1),
                                index = np.array(f[tissue]['barcode']).astype(str),
                                columns = np.array(f[tissue]['ensembl'][tf_idx]).astype(str))

with h5py.File('data/processed/velo.hdf5', 'r') as f:
    velo = pd.DataFrame(np.array(f[tissue]['velo']),
                                index = np.array(f[tissue]['barcode']).astype(str),
                                columns = np.array(f[tissue]['ensembl']).astype(str))

### Read TF activity score matrices. Missing values are treated as zeros.

In [4]:
import os
import pandas as pd

folder = "results/tf_activities_mat"

tf_gene_mat_mean = pd.read_csv(f"{folder}/{tissue}_tfGeneMean.csv", index_col = 0).transpose()
tf_gene_mat_sum = pd.read_csv(f"{folder}/{tissue}_tfGeneSum.csv", index_col = 0).transpose()
tf_gene_mat_mean.fillna(0,inplace = True)
tf_gene_mat_sum.fillna(0,inplace = True)

# scale the value by log2
tf_gene_mat_mean = np.log2(tf_gene_mat_mean+1)
tf_gene_mat_sum = np.log2(tf_gene_mat_sum+1)

### Reformat the data matrices to keep the consistent TFs and genes.
Make rpkm matrix and tf activity matrix have the same tfs  
Make velo matrix and tf activity matrix have the same genes

In [7]:
# Find common TF 
tfmat_cols = pd.DataFrame(tf_gene_mat_mean.columns.values,columns = ['tfmat_cols'])
comm_tfs = ensembl_to_symbol.merge(how = 'left',
                                    left_on = 'gene_symbol',
                                    right_on = "tfmat_cols",
                                    right = tfmat_cols)
comm_tfs = comm_tfs.loc[~comm_tfs['tfmat_cols'].isna(),]
comm_tfs = comm_tfs.loc[comm_tfs["ensembl_id"].isin(rpkm.columns),]

# Find common (velocity) genes
tfmat_rows = pd.DataFrame(tf_gene_mat_mean.index.values,columns = ['tfmat_rows'])
comm_genes = ensembl_to_symbol.merge(how = 'left',
                                        left_on = 'ensembl_id',
                                        right_on = "tfmat_rows",
                                        right = tfmat_rows)
comm_genes = comm_genes.loc[~comm_genes['tfmat_rows'].isna(),]
comm_genes = comm_genes.loc[comm_genes["ensembl_id"].isin(velo.columns),]

'''
Reorder rpkm mat, velo mat, and cluster label vectors. The TF-Gene matrix follow the same tf_order and gene_order. In this order, the columns in upper left
corner are the tfs commonly found in rpkm matrix and tf activity matrix and the rows represent the genes commonly found in velo matrix
and tf activity matrix.
          TF
       _______ __
      | common|  |
Genes |_______|  |
      |__________|
'''
tf_order = pd.concat([comm_tfs['ensembl_id'], pd.Series(np.setdiff1d(rpkm.columns, comm_tfs['ensembl_id']))])
gene_order = pd.concat([comm_genes['ensembl_id'], pd.Series(np.setdiff1d(velo.columns, comm_genes['ensembl_id']))])

rpkm = rpkm.loc[:,tf_order]
velo = velo.loc[:,gene_order]

act_mat_sum = np.zeros((velo.shape[1], rpkm.shape[1]))
act_mat_mean = np.zeros((velo.shape[1], rpkm.shape[1]))

act_mat_sum[:comm_genes.shape[0],:][:,:comm_tfs.shape[0]] = tf_gene_mat_sum.loc[comm_genes['tfmat_rows'],comm_tfs['tfmat_cols']]
act_mat_mean[:comm_genes.shape[0],:][:,:comm_tfs.shape[0]] = tf_gene_mat_mean.loc[comm_genes['tfmat_rows'],comm_tfs['tfmat_cols']]

act_mat_sum = pd.DataFrame(act_mat_sum, index = gene_order, columns = tf_order)
act_mat_mean = pd.DataFrame(act_mat_mean, index = gene_order, columns = tf_order)

del tf_gene_mat_mean
del tf_gene_mat_sum

### Filter genes
we get genes with more than 4000 cells for which the velocity values are available.

In [10]:
# Get cell counts for which the velocity values are available
n_cells = 4000
cell_counts = (~velo.isna()).sum(axis = 0)
selected_genes = cell_counts.index[cell_counts > 4000].values
print(f"{tissue} has {selected_genes.shape[0]} selected genes.")

Liver has 77 selected genes.


### Standardize data

In [11]:
# standardize rpkm train
rpkm = (rpkm - rpkm.mean())/rpkm.std()

# Fill na as zeros in rpkm matrices
rpkm.fillna(value = 0, inplace = True)

### Compute shap values from trained models

**Define model architecture here**

In [12]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras import regularizers
from tensorflow.keras.layers import concatenate
from tensorflow.keras import Model
import tensorflow as tf

def build_model(num_tf, num_feature_type):
    inputs = [Input(shape = (num_feature_type,)) for i in range(num_tf)]
    concat_layer = [Dense(1, activation = "relu", kernel_regularizer = tf.keras.regularizers.l1(0.01))(inp) for inp in inputs]
    concat_layer = concatenate(concat_layer)
    out = Dense(64, activation = "relu", kernel_regularizer = tf.keras.regularizers.l1(0.01))(concat_layer)
    out = Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.l1(0.01))(out)
    out = Dense(16, activation = "relu", kernel_regularizer = tf.keras.regularizers.l1(0.01))(out)
    out = Dense(1, activation = "linear")(out)

    model = Model(inputs = inputs, outputs = out)
    model.compile()
    return(model)

**Load trained model weights and compute SHAP values**  
- set `num_feature_type` as 3 for current framework
- set `n_jobs` to your perferred number. Should be less than total number of CPU cores.
- `n_rep` is equal to the number used in `MTLRANK_train.ipynb`

In [13]:
num_feature_type = 3
n_jobs = 20
n_rep = 5

In [18]:
from time import time
import shap
from joblib import Parallel,delayed

iterator = [] 
for gene in selected_genes:
    for rep in range(1,n_rep+1):
        iterator.append([gene,rep])

def run_shap(tissue, gene, rep):

    begin = time()

    # Build, compile the model, and load up weights from trained model.
    model = build_model(num_tf = rpkm.shape[1], num_feature_type = num_feature_type)
    model.load_weights(f"results/full_model/{gene}-{tissue}-rep{rep}")

    # Prepare inputs
    velo_train = velo[gene]

    # Remove NAs.
    select = ~pd.isna(velo_train)
    rpkm_train = rpkm.loc[select,:]

    X_train = []

    # Generate inputs for current gene
    for i,TF in enumerate(rpkm_train.columns):

        X_train.append(np.empty((rpkm_train.shape[0], num_feature_type)))

        # For current tf, get expression rpkms
        X_train[-1][:,0] = rpkm_train[TF].values

        # Get TF mean signals and TF sum signals
        X_train[-1][:,1] = act_mat_mean.loc[gene,TF]
        X_train[-1][:,2] = act_mat_sum.loc[gene,TF]

    # Zeros as reference samples
    background = [np.zeros((1,3)) for TF in rpkm.columns]

    # Compute shap values for all samples
    e = shap.DeepExplainer(model, background)
    shap_values = e.shap_values(X_train)

    # Aggregate shap values (absolute) for each feature (TF)
    tf_shap_values = np.array([np.abs(np.abs(TF.sum(axis = 0))).sum() for TF in shap_values[0]])
    idx = tf_shap_values.argsort()[::-1] # Descending order
    tf_shap_values = tf_shap_values[idx]
    tf_ensembl = rpkm.columns[idx]
    ranking = pd.DataFrame({"gene_ensembl":gene,"tf_ensembl":tf_ensembl,"sum_shap_value":tf_shap_values,"rep":rep})
    
    end = time()
    print(f"Gene:{gene}, rep:{rep}, completed; {end-begin}s used")
    return(ranking)

# Run shap computation in parallel
res = Parallel(n_jobs = n_jobs)(delayed(run_shap)(tissue, gene, rep)
                              for gene,rep in iterator)

2022-12-13 10:42:08.066225: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 10:42:08.084406: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 10:42:08.223472: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable th

Completed 295.3040544986725s used
Completed 305.1210994720459s used
Completed 294.3556139469147s used


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.


Completed 292.66400480270386s used
Completed 296.1154396533966s used
Completed 307.86273169517517s used


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
2022-12-13 10:57:09.664178: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 10:57:11.828947: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructi

...

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Completed 307.9920892715454s used


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Completed 312.1095435619354s used


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Save TF ranking results

In [21]:
# Create folder for saving ranking results
if not os.path.exists("results/ranking"):
    os.mkdir("results/ranking")
        
# Save TF ranking results
ranking = pd.concat(res) 
ranking.to_csv(f"results/ranking/shap-val-{tissue}.csv",header = True, index = False)